# CEIA


## Procesamiento de lenguaje natural
### Bot con Spacy utilizando un corpus de wikipedia de Argentinos Juniors

#### Para poder utilizar el algoritmo en Español se utiliza la libería Spacy-Stanza

##### CONCLUSIONES: No se pudo lograr volver a procesar con el algoritmo Spacy-Stanza ("nlp()") solamente el texto ingresado por el usuario. Al tener que volver a procesar todo el texto el sistema es muy lento.


In [62]:
import json
import string
import random
import re
import urllib.request

import numpy as np

# Para leer y parsear el texto en HTML de wikipedia
import bs4 as bs


In [3]:
# La última versión de spacy-stanza (>1.0) es compatible solo con spacy >=3.0
# Nota: spacy 3.0 incorpora al pepiline nlp transformers
!pip install -U spacy==3.1 --quiet
!pip install -U spacy-stanza==1.0.0 --quiet

     |████████████████████████████████| 6.4 MB 5.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.3.0 requires spacy<3.4.0,>=3.3.0.dev0, but you have spacy 3.1.0 which is incompatible.
     |████████████████████████████████| 342 kB 6.8 MB/s 


In [4]:
import stanza
import spacy_stanza
# Vamos a usar SpaCy-Stanza. Stanza es una librería de NLP de Stanford
# SpaCy armó un wrapper para los pipelines y modelos de Stanza
# https://stanfordnlp.github.io/stanza/

# Descargar el diccionario en español y armar el pipeline de NLP con spacy. 
stanza.download("es")
nlp = spacy_stanza.load_pipeline("es")

2022-07-07 12:53:50 INFO: Downloading default packages for language: es (Spanish)...


2022-07-07 12:55:42 INFO: Finished downloading models and saved to /root/stanza_resources.
2022-07-07 12:55:42 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

2022-07-07 12:55:42 INFO: Use device: cpu
2022-07-07 12:55:42 INFO: Loading: tokenize
2022-07-07 12:55:42 INFO: Loading: mwt
2022-07-07 12:55:42 INFO: Loading: pos
2022-07-07 12:55:42 INFO: Loading: lemma
2022-07-07 12:55:42 INFO: Loading: depparse
2022-07-07 12:55:42 INFO: Loading: ner
2022-07-07 12:55:44 INFO: Done loading processors!


In [5]:
import re
import unicodedata

# El preprocesamento en castellano requiere más trabajo

# Referencia de regex:
# https://docs.python.org/3/library/re.html

def preprocess_clean_text(text):    
    # sacar tildes de las palabras
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    # quitar caracteres especiales
    pattern = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
    text = re.sub(pattern, '', text)
    pattern = r'[^a-zA-z.,!?/:;\"\'\s]' 
    # quitar números
    text = re.sub(pattern, '', text)
    # quitar caracteres de puntiación
    text = ''.join([c for c in text if c not in string.punctuation])
    return text

### Datos
Se consumira los datos del artículo de wikipedia sobre el club Argentinos Juniors

In [6]:
raw_html = urllib.request.urlopen('https://es.wikipedia.org/wiki/Asociaci%C3%B3n_Atl%C3%A9tica_Argentinos_Juniors')
raw_html = raw_html.read()

article_html = bs.BeautifulSoup(raw_html, 'lxml')

article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text

article_text = article_text.lower()

In [7]:
# Demos un vistazo
article_text

'la asociación atlética argentinos juniors, conocida como argentinos juniors o simplemente argentinos, es una grande centenaria institución polideportiva fundada el 15 de agosto de 1904 en el barrio de villa crespo, ciudad autónoma de buenos aires, argentina. participa en la liga profesional de fútbol argentino. para mayo del 2022 el club poseia 13.799 socios. \nsu disciplina más destacada es el fútbol, en el que es reconocido por la fifa como uno de los 11 clubes clásicos de la argentina,[1]\u200b siendo reconocido ante todo por ser un club formador de numerosos futbolistas de proyección internacional como diego armando maradona, fernando redondo, juan román riquelme, claudio borghi, esteban cambiasso, sergio batista, juan pablo sorin, fernando cáceres, leonel gancedo, diego placente, nicolás pareja, néstor ortigoza, lucas biglia, lucas barrios, federico insúa, leonardo pisculichi, andrés d\'alessandro, fabricio coloccini,esteban rolón, juan ramírez y nicolás iván gonzález (entre otro

In [8]:
print("Cantidad de caracteres en la nota:", len(article_text))

Cantidad de caracteres en la nota: 100469


### 2 - Preprocesamiento
- Remover caracteres especiales
- Quitar espacios o saltos

In [9]:
article_text_preprocesado = preprocess_clean_text(article_text)
article_text_preprocesado 

'la asociacion atletica argentinos juniors conocida como argentinos juniors o simplemente argentinos es una grande centenaria institucion polideportiva fundada el  de agosto de  en el barrio de villa crespo ciudad autonoma de buenos aires argentina participa en la liga profesional de futbol argentino para mayo del  el club poseia  socios \nsu disciplina mas destacada es el futbol en el que es reconocido por la fifa como uno de los  clubes clasicos de la argentina siendo reconocido ante todo por ser un club formador de numerosos futbolistas de proyeccion internacional como diego armando maradona fernando redondo juan roman riquelme claudio borghi esteban cambiasso sergio batista juan pablo sorin fernando caceres leonel gancedo diego placente nicolas pareja nestor ortigoza lucas biglia lucas barrios federico insua leonardo pisculichi andres dalessandro fabricio colocciniesteban rolon juan ramirez y nicolas ivan gonzalez entre otros por tal motivo es reconocido tanto por la afa como por l

In [10]:
print("Cantidad de caracteres en el texto:", len(article_text_preprocesado))

Cantidad de caracteres en el texto: 96191


### 3 - Dividir el texto en sentencias. Seleccionamos los primeros 10.000 caracteres porque no alcanza la memoria del Colab para el texto entero

In [41]:
doc = nlp(article_text[0:10000])
corpus = [sent.text.strip() for sent in doc.sents]

# corpus = nltk.sent_tokenize(text)
# words = nltk.word_tokenize(text)

/usr/local/lib/python3.7/dist-packages/stanza/models/common/beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Due to multiword token expansion or an alignment issue, the original text has been replaced by space-separated expanded tokens.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['la', 'asociación', '

In [12]:
# Demos un vistazo
corpus[:2]

['la asociación atlética argentinos juniors , conocida como argentinos juniors o simplemente argentinos , es una grande centenaria institución polideportiva fundada el 15 de agosto de 1904 en el barrio de villa crespo , ciudad autónoma de buenos aires , argentina .',
 'participa en la liga profesional de fútbol argentino .']

### 4 - Funciones de ayuda para limpiar y procesar el input del usuario
- Lematizar los tokens de la oración
- Quitar símbolos de puntuación

In [56]:
def perform_lemmatization(tokens):
    return [token.lemma_ for token in tokens]
        # Realiza la lematización
#punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)
def get_processed_text(document):
    # 1 - reduce el texto a mínuscula
    # 2 - quitar los simbolos de puntuacion
    # 3 - realiza la tokenización

    return perform_lemmatization(nlp(document))


### 5 - Utilizar vectores TF-IDF y la similitud coseno construido con el corpus de wikipedia

In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def generate_response(user_input, doc_lemma_):
    response = ''
    # Sumar al corpus la pregunta del usuario para calcular
    # su cercania con otros documentos/sentencias
    #user_input_nlp = nlp(user_input)
    doc_lemma_.append(user_input)

    # Crear un vectorizar TFIDF que quite las "stop words" del ingles y utilice
    # nuestra funcion para obtener los tokens lematizados "get_processed_text"
    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text)  


    # Crear los vectores a partir del corpus
    all_word_vectors = word_vectorizer.fit_transform(doc_lemma_)

    # Calcular la similitud coseno entre todas los documentos excepto el agregado (el útlimo "-1")
    # NOTA: con los word embedings veremos más en detalle esta matriz de similitud
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

    # Obtener el índice del vector más cercano a nuestra oración
    # --> descartando la similitud contra nuestor vector propio
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]
    #vector_matched = 0
    if vector_matched == 0:
        response = "I am sorry, I could not understand you"
    else:
        response = doc_lemma_[similar_sentence_number]
    
    doc_lemma_.remove(user_input)
    return response

### 6 - Ensayar el sistema
El sistema intentará encontrar la parte del artículo que más se relaciona con nuestro texto de entrada. Sugerencias ensayar:
- Grand slam
- tournaments
- nadal
- artificial intelligence

In [60]:
# Se utilizará gradio para ensayar el bot
# Herramienta poderosa para crear interfaces rápidas para ensayar modelos
# https://gradio.app/
import sys
!{sys.executable} -m pip install gradio --quiet

In [61]:
import gradio as gr

def bot_response(human_text):
    print(human_text)
    return generate_response(human_text.lower(), corpus)

iface = gr.Interface(
    fn=bot_response,
    inputs=["textbox"],
    outputs="text",
    layout="vertical")

iface.launch(debug=True)

/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `layout` parameter is deprecated, and it has no effect
  warnings.warn(value)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://24675.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


quien fue el mejor jugador del club?


/usr/local/lib/python3.7/dist-packages/stanza/models/common/beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: Due to multiword token expansion or an alignment issue, the original text has been replaced by space-separated expanded tokens.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['quien', 'fue', 'el', 'mejor', 'jugador', 

quien es el presidente del club?


/usr/local/lib/python3.7/dist-packages/stanza/models/common/beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: Due to multiword token expansion or an alignment issue, the original text has been replaced by space-separated expanded tokens.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['quien', 'es', 'el', 'presidente', 'de', '

presidente de argentinos juniors
fundacion
mejor jugador de argentinos
campeonatos logrados
Keyboard interruption in main thread... closing server.


(<gradio.routes.App at 0x7fd7b427b810>,
 'http://127.0.0.1:7860/',
 'https://24675.gradio.app')

## CONCLUSIONES: No se pudo lograr volver a procesar con el algoritmo Spacy-Stanza ("nlp()") solamente el texto ingresado por el usuario. Al tener que volver a procesar todo el texto el sistema es muy lento.